In [1]:
import pandas as pd
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
super = pd.read_csv("train_supervised_dataset.csv")
super.head()

,id,name,good,brand
0,0,Petmax Бантик леопард с красн розой 2шт,бантик,petmax
1,1,87191 Бусы для елки шарики_87191,бусы,NaN
2,2,Футболка Piazza Italia WR011446881,футболка,piazza italia
3,3,7) YI572-03X-ONE ЗАКОЛКА ДЛЯ ВОЛОС ДЛЯ ДЕВОЧКИ,заколка,NaN
4,4,Одежда (вес) 1500,одежда,NaN


In [3]:
unique_good = set(super.good.dropna().unique().tolist())

In [4]:
test = pd.read_csv("test_dataset.csv")
sub = pd.read_csv("NER_baseline_labse_v3.csv")
test = test.merge(sub, on="id")
test.name = test.name.apply(lambda x: x.lower())
test.head()

,id,name,good,brand
0,0,"469-210 ермак клей универсальный, 15мл, блистер",клей,ермак
1,1,торт сладушка зимняя вишня 700г,торт,сладушка
2,2,"смеситель ""calorie"" 1023 а06 д/кухни",смеситель,calorie
3,3,лимон 50гр бар,лимон,NaN
4,4,"коньяк сараджишвили 5 лет 0,5л грузия",коньяк,сараджишвили


In [7]:
def find_good(text):
    text = text.split()
    for t in text:
        if t in unique_good:
            return t

    return ""

In [8]:
test["find_good"] = test.name.parallel_apply(find_good)
test

,id,name,good,brand,find_good
0,0,"469-210 ермак клей универсальный, 15мл, блистер",клей,ермак,клей
1,1,торт сладушка зимняя вишня 700г,торт,сладушка,торт
2,2,"смеситель ""calorie"" 1023 а06 д/кухни",смеситель,calorie,смеситель
3,3,лимон 50гр бар,лимон,NaN,
4,4,"коньяк сараджишвили 5 лет 0,5л грузия",коньяк,сараджишвили,коньяк
...,...,...,...,...,...
4995,4995,"774352 рамка 2п., сл. кость",рамка,NaN,рамка
4996,4996,энерг. напиток red bull 0.25л,напиток,red bull,напиток
4997,4997,36/025 наконечники (т. никель) шт,наконечники,NaN,
4998,4998,шоколад риттерспорт мол.с цел.миндалем 100г,шоколад,риттерспорт,шоколад


In [15]:
test[(test.good != test.find_good) & (test.find_good != "")]

,id,name,good,brand,find_good
13,13,botavikos крем-гель д/душа боди релакс 200 мл ...,крем,botavikos,боди
53,53,"тархун лимонад 0,4л.",лимонад,тархун,тархун
61,61,"и 40 цифра ""4"" золото в упаковке / four / 1 шт...",NaN,NaN,цифра
62,62,4607149848928 диск отрезной луга 54с по камню ...,диск отрезной,луга,диск
65,65,3390150569975 набор (дезодорант рол.75мл+гел,дезодорант,NaN,набор
...,...,...,...,...,...
4940,4940,подсолнух зеркало,зеркало,NaN,подсолнух
4971,4971,"перекись водорода буфус 3%, 10мл, тюб-кап., n5...",перекись водорода,буфус,р-р
4972,4972,фаленопсис 2 ств. 12*65 микс,фаленопсис,NaN,микс
4993,4993,средство для мытья посуды золушка лимон 1/500мл,средство для мытья посуды,золушка,средство
